In [0]:
# 1. Reach out to the "Bulletin Board" (Task Values).
# We tell it: "Go find the task named 'bronze' and give me the value labeled 'bronze_output'."
bronze_output = dbutils.jobs.taskValues.get(taskKey="Bronze", key="bronze_output")

# 2. Extract the variables from that "package".

# The .get("key", "") is a safety feature: if the key is missing, it returns an empty string instead of crashing.
start_date = bronze_output.get("start_date", "")
end_date = bronze_output.get("end_date", "")
bronze_adls = bronze_output.get("bronze_adls", "")
silver_adls = bronze_output.get("silver_adls", "")
gold_adls = bronze_output.get("gold_adls", "")

# 3. Print the results to verify we have the right "keys" to the storage account.
print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")

In [0]:
# 1. Define the stages of the Medallion Architecture data pipeline
tiers = ["bronze", "silver", "gold"]

# 2. Automatically generate the storage URLs for each tier.
# This creates a dictionary where each tier is a key, and the Azure storage path is the value.
# The 'f' before the string allows us to plug the {tier} variable directly into the URL.
adls_paths = {tier: f"abfss://{tier}@earthquakestorageaccount.dfs.core.windows.net/" for tier in tiers}
adls_paths


# 3. Pull the specific URLs out of our dictionary and save them to easy-to-use variables.
# This is like taking a long address from a directory and writing it on a sticky note.
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

# 4. Use Databricks Utilities (dbutils) to list the files in each folder.
# This confirms that the connection to Azure is working and shows you what data is available.
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta
     
start_date = date.today() - timedelta(1)
end_date = date.today()
start_date, end_date

# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")
#df.head()

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

#df.head()


In [0]:

# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)
df.head()

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)
     

In [0]:
dbutils.jobs.taskValues.set(key = "silver_output", value = silver_output_path) #we link the gold layer to the silver layer with this value  